In [3]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/New Project/balanced_dataset.csv")

# Data Splitting

In [4]:
from sklearn.model_selection import train_test_split

# train (80%), val (10%), test (10%)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print(f"Train: {len(train_df)}, Validation: {len(val_df)}, Test: {len(test_df)}")

Train: 8000, Validation: 1000, Test: 1000


# Tokenising Data

In [5]:
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [7]:
def tokenize_text(texts):
    # Convert any non-string values to strings
    texts = texts.astype(str).tolist()
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

# Tokenize splits
train_encodings = tokenize_text(train_df["text"])
val_encodings = tokenize_text(val_df["text"])
test_encodings = tokenize_text(test_df["text"])

# Creating PyTorch Dataset

In [8]:
import torch
from torch.utils.data import Dataset

In [9]:
class HateSpeechDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": torch.tensor(self.labels[idx])
        }

    def __len__(self):
        return len(self.labels)

# datasets
train_dataset = HateSpeechDataset(train_encodings, train_df["label"].values)
val_dataset = HateSpeechDataset(val_encodings, val_df["label"].values)
test_dataset = HateSpeechDataset(test_encodings, test_df["label"].values)

# Importing **DistilBert** model for classification

In [10]:
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2  # 0 (noHate) and 1 (hate)
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import classification_report

In [12]:
# Define metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    return classification_report(labels, preds, output_dict=True)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    evaluation_strategy="epoch",
    logging_steps=100,
    save_strategy="no"
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# Training the Model

In [13]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: architrawat25 (architrawat25-dehradun) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,0,1,Accuracy,Macro avg,Weighted avg
1,0.249000,0.196036,"{'precision': 0.908745247148289, 'recall': 0.9540918163672655, 'f1-score': 0.9308666017526777, 'support': 501.0}","{'precision': 0.9514767932489452, 'recall': 0.9038076152304609, 'f1-score': 0.9270298047276464, 'support': 499.0}",0.929000,"{'precision': 0.9301110201986171, 'recall': 0.9289497157988632, 'f1-score': 0.928948203240162, 'support': 1000.0}","{'precision': 0.9300682886525165, 'recall': 0.929, 'f1-score': 0.9289520400371871, 'support': 1000.0}"
2,0.140900,0.227384,"{'precision': 0.8970588235294118, 'recall': 0.9740518962075848, 'f1-score': 0.9339712918660287, 'support': 501.0}","{'precision': 0.9714912280701754, 'recall': 0.8877755511022044, 'f1-score': 0.9277486910994764, 'support': 499.0}",0.931000,"{'precision': 0.9342750257997936, 'recall': 0.9309137236548946, 'f1-score': 0.9308599914827526, 'support': 1000.0}","{'precision': 0.9342005933952527, 'recall': 0.931, 'f1-score': 0.9308662140835191, 'support': 1000.0}"
3,0.025400,0.278902,"{'precision': 0.949290060851927, 'recall': 0.9341317365269461, 'f1-score': 0.9416498993963782, 'support': 501.0}","{'precision': 0.9349112426035503, 'recall': 0.9498997995991983, 'f1-score': 0.9423459244532804, 'support': 499.0}",0.942000,"{'precision': 0.9421006517277386, 'recall': 0.9420157680630723, 'f1-score': 0.9419979119248293, 'support': 1000.0}","{'precision': 0.9421150305459871, 'recall': 0.942, 'f1-score': 0.9419972158997723, 'support': 1000.0}"


Trainer is attempting to log a value of "{'precision': 0.908745247148289, 'recall': 0.9540918163672655, 'f1-score': 0.9308666017526777, 'support': 501.0}" of type <class 'dict'> for key "eval/0" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9514767932489452, 'recall': 0.9038076152304609, 'f1-score': 0.9270298047276464, 'support': 499.0}" of type <class 'dict'> for key "eval/1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9301110201986171, 'recall': 0.9289497157988632, 'f1-score': 0.928948203240162, 'support': 1000.0}" of type <class 'dict'> for key "eval/macro avg" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9300682886525165, 're

TrainOutput(global_step=1500, training_loss=0.15615393781661988, metrics={'train_runtime': 343.1395, 'train_samples_per_second': 69.942, 'train_steps_per_second': 4.371, 'total_flos': 794804391936000.0, 'train_loss': 0.15615393781661988, 'epoch': 3.0})

## Classification Report

In [14]:
# Generate predictions
test_results = trainer.predict(test_dataset)

# Extract labels and predictions
y_true = test_results.label_ids
y_pred = test_results.predictions.argmax(-1)


report = classification_report(y_true, y_pred, target_names=["noHate", "hate"])
print(report)

              precision    recall  f1-score   support

      noHate       0.94      0.94      0.94       512
        hate       0.94      0.93      0.94       488

    accuracy                           0.94      1000
   macro avg       0.94      0.94      0.94      1000
weighted avg       0.94      0.94      0.94      1000



# Saving the model

In [25]:
model.save_pretrained("./hate_speech_detector")
tokenizer.save_pretrained("./hate_speech_detector")

# Compress and download
!zip -r /content/hate_speech_detector.zip /content/hate_speech_detector
files.download("/content/hate_speech_detector.zip")

  adding: content/hate_speech_detector/ (stored 0%)
  adding: content/hate_speech_detector/model.safetensors (deflated 8%)
  adding: content/hate_speech_detector/tokenizer_config.json (deflated 75%)
  adding: content/hate_speech_detector/special_tokens_map.json (deflated 42%)
  adding: content/hate_speech_detector/config.json (deflated 46%)
  adding: content/hate_speech_detector/vocab.txt (deflated 53%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Saving on Google drive

In [26]:
model.save_pretrained("/content/drive/MyDrive/hate_speech_detector")
tokenizer.save_pretrained("/content/drive/MyDrive/hate_speech_detector")

('/content/drive/MyDrive/hate_speech_detector/tokenizer_config.json',
 '/content/drive/MyDrive/hate_speech_detector/special_tokens_map.json',
 '/content/drive/MyDrive/hate_speech_detector/vocab.txt',
 '/content/drive/MyDrive/hate_speech_detector/added_tokens.json')

In [27]:
# Verify
!ls "/content/drive/MyDrive/hate_speech_detector"

config.json  model.safetensors	special_tokens_map.json  tokenizer_config.json	vocab.txt
